In [119]:
%matplotlib inline

from __future__ import absolute_import, print_function, unicode_literals, division
from sklearn.datasets import fetch_mldata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
import random as rand
import copy

In [120]:
class MLP(object):
    """
    NOTE: Matrix operations are modified from RBM file
    In particular, we have an an an input of n data points with
    dimension d as a d-by-n matrix
    
    """
    def __init__(self, data, num_hidden, learn_rate):
        """
        Target values in training data are the first row of data 
        """
        self.targets = data[0]
        self.data = data[1:,:]
        self.num_data = data.shape[1]
        
        self.num_visible = data.shape[0] - 1
        self.num_hidden = num_hidden
        self.learn_rate = learn_rate
        self.state_size = 2 * (np.amax(self.targets) - np.amin(self.targets)) + 4

        self.weights = 2 * np.random.rand(self.num_visible + 1, self.num_hidden) - 1
        self.data = np.insert(self.data, 0, 1, axis = 0)
    
    def _sigmoid(self,x):
        return 1.0/(1.0+np.exp(-x))
        
    def _activated(self,mat):
        activated = self._sigmoid(self.weights.T.dot(mat))
        return activated
    
    def _gradient(self,mat):
        prod = mat - np.multiply(mat,mat)
        return prod
    
    def train(self, iterations=30):
        data_copy = copy.deepcopy(self.data)
        for j in range(0,iterations):
            for i in range(0,self.num_data):

                """Feed-forward phase"""
                data_col = data_copy[:,i]
                data_col = np.reshape(data_col, (-1, 1))
                activated_output = (self._activated(data_col))[0]
                guess = self.state_size * (2 * activated_output - 1)

                """Error calculation phase"""
                error = self.targets[i] - guess

                """Back-propagation phase"""
                grad = (self._gradient(activated_output))[0]
                c = (error * self.learn_rate)[0]
                wt_change = c * grad * data_col
                self.weights = wt_change + self.weights
    
    def pred(self, test):
        """
        No target values for prediction data so there is one fewer row
        than in the training data
        """
        test_copy = copy.deepcopy(test)
        test_copy = np.insert(test_copy, 0, 1, axis = 0)
        pred_p = self._activated(test_copy)[0]
        pred = 2 * pred_p - 1
        pred = self.state_size * pred
        return pred


In [121]:
# Reference links:
# - www.cse.unsw.edu.au/~cs9417ml/MLP2/
# - www.hiit.fi/u/ahonkela/dippa/node41.html

# Generate training data (no noise)
dims = 10
data_pts = 50 * dims
sample_data = 2*np.random.rand(dims,data_pts)-1
sample_data[0] = (4 * sample_data[1] - 1 * sample_data[2]) 

# Need to write binary target values for data as well for MLP
MLP_1 = MLP(sample_data,1,0.01) #0.035 threshold
print("Weights^T: \n",MLP_1.weights.T)

Weights^T: 
 [[-0.70816771 -0.29493128  0.8096765   0.30764328  0.14108006 -0.74447699
  -0.36070218 -0.01232617  0.14024816 -0.37432374]]


In [122]:
MLP_1.train(10)
print("Weights: \n", MLP_1.weights)

Weights: 
 [[  1.79901080e-05]
 [  3.53444997e-01]
 [ -8.87793129e-02]
 [ -5.86038602e-05]
 [  1.05837317e-05]
 [  7.79664335e-05]
 [  1.75605806e-05]
 [ -6.72653392e-05]
 [ -7.55286328e-05]
 [ -6.27946120e-05]]


In [123]:
dims = 9
test_pts = 10
test_data = 2*np.random.rand(dims,test_pts)-1
print("targets: \n", 4 * test_data[0] - 1 * test_data[1] )
guesses = MLP_1.pred(test_data)
print("Guesses: \n", guesses)

targets: 
 [ 3.8062859  -2.63614404 -3.54867594  2.04171831  1.10545327 -0.30439306
  1.8436519  -3.29094546 -0.4100905   1.63184036]
Guesses: 
 [ 3.80039586 -2.64454947 -3.54461537  2.05550002  1.11560882 -0.30252534
  1.85207075 -3.29144664 -0.41268897  1.64072505]


In [128]:
# Training on XOR will not yield results since single
# perceptron cannot resolve nonlinearity
data2 = np.array([[0,1,1,0],[0,0,1,1],[0,1,0,1]])
MLP_2 = MLP(data2,1,0.01)
MLP_2.train(1000)
data2a = np.array([[0,0,1,1],[0,1,0,1]])
MLP_2.pred(data2a)

array([ 0.50858904,  0.50044083,  0.49750052,  0.48934979])